# Intake-STAC + Holoviz Visualizations

Intake-STAC takes advantage of Intake's built-in graphical user-interface (GUI), which can browse catalogs and display pre-defined interactive plots of assets. This interface is built using the [Panel](https://panel.holoviz.org) library, which is part of the [Holoviz](https://holoviz.org) collection of libraries for browser-based visualizations in Python.

In [ ]:
%matplotlib inline
import intake

In [ ]:
# Load results from previous satsearch (see examples/aws-earth-search.ipynb )
from satstac import ItemCollection
items = ItemCollection.open('my-s2-l2a-cogs.json')

# Intake-STAC built-in GUI

Currently you can choose a `thumbnail` plot for RGB thumbs. Or a `geotiff` plot for dynamically-updating full resultion single band images (Warning, currently initializing plot can be slow for large geotiffs, likely can improve this in the future!).

In [ ]:
catalog = intake.open_stac_item_collection(items)

In [ ]:
# NOTE: import statement required for dynamic=True
import hvplot.xarray
intake.gui.add(catalog)
intake.gui

In [ ]:
# Here's how you would load plots outside of the GUI
item = catalog['S2A_36MYB_20200814_0_L2A']
thumb = item['thumbnail']
da = thumb.to_dask()
da.hvplot.rgb(x='x', y='y', bands='channel',
              title=item.name,
              data_aspect=1, flip_yaxis=True, xaxis=False, yaxis=None)

In [ ]:
# Interactive plot with hvplot and datashader (dynamically-updated resolution)
da = item['B06'].to_dask()
da.sel(band=1).hvplot.image(x='x', y='y',
                title=f"{item.name} - {item['B06'].description}",
                rasterize=True, dynamic=True,
                frame_width=500, data_aspect=1)

## ipywidgets

[ipywidgets](https://ipywidgets.readthedocs.io/en/stable/) provide another convenient approach to custom visualizations. The function below allows us to browse through all the image thumbnails for a group of images.

In [ ]:
from ipywidgets import interact
from IPython.display import display, Image

def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))
    
    interact(view_image, i=(0,n-1))

browse_images(items)